# 🎯 Object Detection: YOLO & Faster R-CNN

**Author**: Data Science Master System  
**Difficulty**: ⭐⭐⭐ Advanced  
**Time**: 75 minutes  
**Prerequisites**: 11_cv_image_classification, PyTorch

## Learning Objectives
- Understand object detection fundamentals
- Implement YOLO for real-time detection
- Use Faster R-CNN for high accuracy
- Compare one-stage vs two-stage detectors

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

## 1. YOLOv8 (Ultralytics)

In [ ]:
try:
    from ultralytics import YOLO
    
    # Load pretrained YOLOv8
    model = YOLO('yolov8n.pt')  # Nano model (fastest)
    print(f"✅ YOLOv8 loaded")
    
    # Model variants
    variants = {
        'yolov8n': ('Nano', '3.2M', '37.3 mAP'),
        'yolov8s': ('Small', '11.2M', '44.9 mAP'),
        'yolov8m': ('Medium', '25.9M', '50.2 mAP'),
        'yolov8l': ('Large', '43.7M', '52.9 mAP'),
        'yolov8x': ('XLarge', '68.2M', '53.9 mAP'),
    }
    
    print("\n📊 YOLO Variants:")
    for name, (size, params, acc) in variants.items():
        print(f"  {name}: {size}, {params} params, {acc}")
        
except ImportError:
    print("Install: pip install ultralytics")

## 2. YOLO Inference

In [ ]:
# Create sample image
sample_image = np.random.randint(0, 255, (640, 640, 3), dtype=np.uint8)

try:
    # Run inference
    results = model.predict(sample_image, conf=0.5, verbose=False)
    
    # Parse results
    for r in results:
        boxes = r.boxes.xyxy.cpu().numpy()  # Bounding boxes
        classes = r.boxes.cls.cpu().numpy()  # Class IDs
        confs = r.boxes.conf.cpu().numpy()   # Confidence scores
        
        print(f"Detected {len(boxes)} objects")
        
except Exception as e:
    print(f"Inference demo: {e}")

## 3. Faster R-CNN (TorchVision)

In [ ]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights

# Load pretrained
weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
frcnn = fasterrcnn_resnet50_fpn(weights=weights).to(device)
frcnn.eval()

COCO_CLASSES = weights.meta['categories']
print(f"✅ Faster R-CNN loaded")
print(f"Classes: {len(COCO_CLASSES)} COCO categories")

In [ ]:
# Inference with Faster R-CNN
def detect_objects(model, image, threshold=0.5):
    """Run object detection."""
    from torchvision import transforms
    
    transform = transforms.Compose([transforms.ToTensor()])
    img_tensor = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        predictions = model(img_tensor)[0]
    
    # Filter by confidence
    mask = predictions['scores'] > threshold
    return {
        'boxes': predictions['boxes'][mask].cpu().numpy(),
        'labels': predictions['labels'][mask].cpu().numpy(),
        'scores': predictions['scores'][mask].cpu().numpy()
    }

# Test
test_img = Image.fromarray(np.random.randint(0, 255, (224, 224, 3), dtype=np.uint8))
detections = detect_objects(frcnn, test_img)
print(f"Detected: {len(detections['boxes'])} objects")

## 4. Model Comparison

In [ ]:
import pandas as pd

comparison = pd.DataFrame({
    'Model': ['YOLOv8n', 'YOLOv8m', 'Faster R-CNN', 'RetinaNet', 'DETR'],
    'Type': ['One-stage', 'One-stage', 'Two-stage', 'One-stage', 'Transformer'],
    'mAP': [37.3, 50.2, 42.0, 40.4, 42.0],
    'FPS': [100, 40, 8, 15, 5],
    'Best For': ['Real-time', 'Balanced', 'Accuracy', 'Small objects', 'Research']
})

print("📊 Object Detection Models:")
display(comparison)

## 5. Custom Training

In [ ]:
# YOLO custom training example
training_code = '''
from ultralytics import YOLO

# Load base model
model = YOLO('yolov8n.pt')

# Train on custom dataset
model.train(
    data='custom_dataset.yaml',  # Dataset config
    epochs=100,
    imgsz=640,
    batch=16,
    device=0  # GPU
)

# Export for deployment
model.export(format='onnx')  # or 'tflite', 'tensorrt'
'''
print("📋 Custom Training:")
print(training_code)

## 6. Deployment & Cost Analysis

In [ ]:
costs = pd.DataFrame({
    'Platform': ['AWS Lambda', 'GCP Cloud Run', 'Edge (Jetson)', 'Local GPU'],
    'Model': ['YOLOv8n', 'YOLOv8s', 'YOLOv8n', 'YOLOv8l'],
    'Latency': ['~500ms', '~200ms', '~30ms', '~10ms'],
    'Cost/1M Images': ['$200', '$150', '$50 (hardware)', '$100 (power)']
})

print("💰 Deployment Costs:")
display(costs)

## 🎯 Key Takeaways
1. **YOLO**: Best for real-time (30+ FPS)
2. **Faster R-CNN**: Best accuracy, slower
3. **Choose based on**: Speed vs accuracy tradeoff
4. **Export**: ONNX for cross-platform deployment

**Next**: 13_cv_image_segmentation.ipynb